In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import datetime
from time import strftime
import random
import time

In [19]:
dict_months = {'01': 'января', '02': 'февраля', '03': 'марта', '04': 'апреля', '05': 'мая', '06': 'июня', '07': 'июля', '08': 'августа', '09': 'сентября', '10': 'октября', '11': 'ноября', '12': 'декабря'}
input_data = pd.read_excel('input_data.xls')
date = input_data['Дата'][0].strftime('%d.%m.%Y')[1:].split('.')
date[1] = dict_months[date[1]]

data_for_writing = pd.DataFrame(columns=['Банк', 'Остаток на счете'])

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome("C:/ProgramData/Anaconda3/Library/bin/chromedriver.exe", options=chrome_options) # Указать свой патч к chromedriver
url = 'http://www.cbr.ru/credit/main.asp'

for bank in input_data['Банки'].dropna():
    driver.get(url) # Загружаем ссылку в Chrome
    form = driver.find_element_by_name("find") # Находим поле для ввода названия банка
    form.clear() # Очищаем поле, если там что-то есть
    form.send_keys(bank) # Вводим нужный банк
    time.sleep(0.5)
    driver.find_element_by_css_selector(".submit").click() # Клик по кнопке Найти
    time.sleep(1)
    try:
        table2 = driver.find_element_by_css_selector(".CBRTBL.info_table").get_attribute('outerHTML') # Получение таблицы
        df2  = pd.read_html(table2)[0] # Запись табилцы в pandas
        for i in df2.index:
            if df2[1][i] == 'Б П С' or df2[1][i] == 'П С':
                k = i
        k += 1
        driver.find_element_by_xpath('//*[@id="content"]//table//tbody//tr[' + str(k) + ']//td[5]//a').click() 
    except:
        data_this = pd.DataFrame([[bank, 'Не удалось найти банк']], columns=['Банк', 'Остаток на счете'])
        data_for_writing = data_for_writing.append(data_this)
        continue
    time.sleep(1)
    try:
        driver.find_element_by_xpath('//*[@id="f_101"]//span[contains(text(), "' + str(date[2]) + '")]').click() # Выбор года
    except:
        data_this = pd.DataFrame([[bank, 'Не удалось найти  информацию по банку']], columns=['Банк', 'Остаток на счете'])
        data_for_writing = data_for_writing.append(data_this)
        continue
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="f_101"]//ul//a[contains(text(), "на ' + str(date[0]) + ' ' + str(date[1]) + '")]').click() # Выбор месяца
    table = driver.find_element_by_css_selector(".data.lined").get_attribute('outerHTML') # Получение таблицы

    df  = pd.read_html(table) # Запись табилцы в pandas
    data = df[0][5:]
    data.reset_index(inplace=True, drop=True)

    sum_residues = 0
    for index in data.index:
        try:
            if int(data[0][index]) in np.array(input_data['Счета']):
                sum_residues += int(data[10][index].replace(u'\xa0', ''))
        except:
            pass

    data_this = pd.DataFrame([[bank, sum_residues]], columns=['Банк', 'Остаток на счете'])
    data_for_writing = data_for_writing.append(data_this)

driver.quit()


data_for_writing.reset_index(inplace=True, drop=True)
data_for_writing.to_excel('Final.xls')